<div style="text-align: center;">  
    <img src="./assets/force_formula.png" style="width: 20%; height: auto;"/>  
</div>

In [1]:
import math
f1=100
f2=290
a=350/2
z=165

f3 = (f2/math.sqrt(2) - f1)*2*math.sqrt(a**2+z**2)/(a-z)
print('f3', f3)

stiffness1 = 771  # N/m    
stiffness2 = 2000
stiffness3 = 67000

length1 = 0.35
length2 = 0.35/2*math.sqrt(2)
length3 = 0.35/2*math.sqrt(2)

dx1 = f1/stiffness1
dx2 = f2/stiffness2
dx3 = f3/stiffness3
ori_length1 = abs(dx1-length1)
ori_length2 = abs(dx2-length2)
ori_length3 = abs(dx3-length3)
print('original length1, length2, length3',ori_length1, ori_length2, ori_length3)

f3 5053.858395660793
original length1, length2, length3 0.22029831387808038 0.10248737341529165 0.17205665109199625


In [2]:
240**3

13824000

In [ ]:
import pickle
import mujoco as mj

export_xml_file_path = '../data/xml/'
export_xml_file_name = "TT12_0823.xml"
xml_path = export_xml_file_path + export_xml_file_name
model = mj.MjModel.from_xml_path(xml_path)  # MuJoCo model
data = mj.MjData(model)


In [ ]:
(1545.6+538.16+3209.02-150-47)/6    # 150是yyf自己买的相机、和木板
1200-362
150/6-8/6   # yyf返还每个人的（除了tyh）
8+23.67 # yyf返还tyh的

In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt  

def plot_data_from_csv(data):  
    time_data = data['Time']  
    
    num_subplots = data.shape[1] - 1  # 减去时间列  
    fig, axs = plt.subplots(num_subplots, 1, figsize=(5, 3*num_subplots))  

    for i, column in enumerate(data.columns[1:]):  
        y_data = data[column]  
        axs[i].grid(True)  
        axs[i].plot(time_data, y_data, label=column)  
        axs[i].set_xlabel('Time')  
        axs[i].set_ylabel(column)  
        # axs[i].set_ylim(-5, +5) 
        axs[i].legend()  

    plt.tight_layout()  
    plt.show()  


In [ ]:
data = pd.read_csv('../data/csv/data_tt12bar_0824.csv')  
time_data = data['Time']
num_subplots = data.shape[1] - 1  # 减去时间列 
plot_data_from_csv(data)

In [ ]:
data = pd.read_csv('../data/csv/data_tt12bar_0824_1.csv')  
time_data = data['Time']
num_subplots = data.shape[1] - 1
plot_data_from_csv(data)

## 拟合合适的碰撞参数

In [ ]:
import pandas as pd
xlsx_path = './drop_test_0826.xlsx'
df = pd.read_excel(xlsx_path, sheet_name='Sheet1')
# 将df的名字为h的列全部乘以0.001
# 截取df的t>0.4的行
adams_df = df[(df['t'] > 0.4) & (df['t'] < 0.6)].copy()
adams_df['h'] = adams_df['h'] * 0.001
adams_np = adams_df.to_numpy()

In [ ]:
adams_np

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(adams_np[:,0], adams_np[:,1])
plt.grid()


In [ ]:
import mujoco as mj

stiffness = 3300
damping = 40
d0 = 0.9
d_width = 0.95
width = 0.001

# stiffness = 5773
# damping = 68
# d0 = 0.26
# d_width = 0.07
# width = 0.065

friction1 = 0.6
friction2 = 0.005
friction3 = 0.0001

a0 = 10
Hz = 1000
BLOCK_HALF_WIDTH = 0.05
mass = 1
time = 0.6

pos = np.array([0, 0, 0.05])
key1pos = np.array([0, 0, 1])
quat = np.array([0, 0, 0, 1])
vel = np.array([0, 0, 0])
angvel = np.array([0, 0, 0])

time_list = []
xpos_list = []
xvel_list = []

xml_string=f"""
            <mujoco>
            <option integrator="implicit" 
                o_solref="{-stiffness} {-damping}" 
                o_solimp="{d0} {d_width} {width} 0.5 1" 
                o_friction = '{friction1} {friction2} {friction3}'
                timestep="{1/Hz}" 
                gravity="0 0 -{a0}" >
            <flag override="disable" />
            </option>
            <worldbody>
                <light name="light_-2"  pos="-2 -1 3" dir="0.666667 0.333333 -0.666667"/>
                <light name="light_2"  pos="2 -1 3" dir="-0.666667 0.333333 -0.666667"/>
                <geom name='floor' 
                    type='plane' 
                    size='0 0 0.01' 
                    priority='1' 
                    solref="{-stiffness} {-damping}"
                    solimp="{d0} {d_width} {width} 0.5 2" 
                    friction='{friction1} {friction2} {friction3}'/>
                <body name='box' pos="{pos[0]} {pos[1]} {pos[2]}" quat='{quat[0]} {quat[1]} {quat[2]} {quat[3]}'>
                    <freejoint/>
                    <inertial pos='0 0 0' mass='0.37' diaginertia='0.00081 0.00081 0.00081'/>
                    <geom name='box' 
                        type='box' 
                        size="{BLOCK_HALF_WIDTH} {BLOCK_HALF_WIDTH} {BLOCK_HALF_WIDTH}" 
                        rgba="0.8 0.1 0.1 1"
                        mass="{mass}"/>
                </body>
            </worldbody>
            <keyframe>
                <key name='initial_state' qvel='{vel[0]} {vel[1]} {vel[2]} {angvel[0]} {angvel[1]} {angvel[2]}'/>
                <key name='initial_state1' qpos='{key1pos[0]} {key1pos[1]} {key1pos[2]} 0 0 0 1' qvel='{vel[0]} {vel[1]} {vel[2]} {angvel[0]} {angvel[1]} {angvel[2]}'/>
            </keyframe>
            </mujoco>
            """

def sim(keyframe_id=0):
    m = mj.MjModel.from_xml_string(xml_string)
    d = mj.MjData(m)
    mj.mj_resetDataKeyframe(m, d, keyframe_id)
    mj.mj_forward(m, d)
    while True:
        if d.time >= time:
            break
        mj.mj_step(m, d)
        time_list.append(d.time)
        xpos_list.append(d.body('box').xpos[2].copy())
        xvel_list.append(d.body('box').cvel[5].copy())

def plot_result(): 
    plt.subplot(2, 1, 1)
    plt.title('Position')
    plt.xlabel('Time')
    plt.ylabel('Position')
    plt.grid()
    plt.plot(time_list, xpos_list)
    plt.subplot(2, 1, 2)
    plt.title('Velocity')
    plt.plot(time_list, xvel_list)
    plt.xlabel('Time')
    plt.ylabel('Velocity')
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
sim(keyframe_id=1)
plot_result()

In [ ]:
# 将time_list和xpos_list转换成numpy数组
mj_np = np.column_stack((np.array(time_list), np.array(xpos_list)))  
mj_np = mj_np[(mj_np[:, 0] > 0.4) & (mj_np[:, 0] < 0.6)]  

In [ ]:
plt.plot(mj_np[:,0], mj_np[:,1])
plt.grid()

In [ ]:
time_a = adams_np[:, 0]  
values_a = adams_np[:, 1]  

time_b = mj_np[:, 0]  
values_b = mj_np[:, 1]  

# 使用 np.interp 进行插值，将 b 的值插值到 a 的时间点  
interpolated_values_b = np.interp(time_a, time_b, values_b)  

absolute_difference = np.abs(values_a - interpolated_values_b)  
mean_absolute_difference = np.sum(absolute_difference, axis=0) 
print(mean_absolute_difference)

In [ ]:
def loss_function(d0=0.9, d_width=0.95, width=0.001, stiffness=3300, damping=40):
    friction1 = 0.6
    friction2 = 0.005
    friction3 = 0.0001

    a0 = 10
    Hz = 1000
    BLOCK_HALF_WIDTH = 0.05
    mass = 1
    time = 0.6

    pos = np.array([0, 0, 0.05])
    key1pos = np.array([0, 0, 1])
    quat = np.array([0, 0, 0, 1])
    vel = np.array([0, 0, 0])
    angvel = np.array([0, 0, 0])

    time_list = []
    xpos_list = []
    
    xml_string=f"""
        <mujoco>
        <option integrator="implicit" 
            o_solref="{-stiffness} {-damping}" 
            o_solimp="{d0} {d_width} {width} 0.5 1" 
            o_friction = '{friction1} {friction2} {friction3}'
            timestep="{1/Hz}" 
            gravity="0 0 -{a0}" >
        <flag override="disable" />
        </option>
        <worldbody>
            <light name="light_-2"  pos="-2 -1 3" dir="0.666667 0.333333 -0.666667"/>
            <light name="light_2"  pos="2 -1 3" dir="-0.666667 0.333333 -0.666667"/>
            <geom name='floor' 
                type='plane' 
                size='0 0 0.01' 
                priority='1' 
                solref="{-stiffness} {-damping}"
                solimp="{d0} {d_width} {width} 0.5 2" 
                friction='{friction1} {friction2} {friction3}'/>
            <body name='box' pos="{pos[0]} {pos[1]} {pos[2]}" quat='{quat[0]} {quat[1]} {quat[2]} {quat[3]}'>
                <freejoint/>
                <inertial pos='0 0 0' mass='0.37' diaginertia='0.00081 0.00081 0.00081'/>
                <geom name='box' 
                    type='box' 
                    size="{BLOCK_HALF_WIDTH} {BLOCK_HALF_WIDTH} {BLOCK_HALF_WIDTH}" 
                    rgba="0.8 0.1 0.1 1"
                    mass="{mass}"/>
            </body>
        </worldbody>
        <keyframe>
            <key name='initial_state' qvel='{vel[0]} {vel[1]} {vel[2]} {angvel[0]} {angvel[1]} {angvel[2]}'/>
            <key name='initial_state1' qpos='{key1pos[0]} {key1pos[1]} {key1pos[2]} 0 0 0 1' qvel='{vel[0]} {vel[1]} {vel[2]} {angvel[0]} {angvel[1]} {angvel[2]}'/>
        </keyframe>
        </mujoco>
        """
    m = mj.MjModel.from_xml_string(xml_string)
    d = mj.MjData(m)
    mj.mj_resetDataKeyframe(m, d, 1)
    mj.mj_forward(m, d)
    while True:
        if d.time >= time:
            break
        mj.mj_step(m, d)
        time_list.append(d.time)
        xpos_list.append(d.body('box').xpos[2].copy())
    mj_np = np.column_stack((np.array(time_list), np.array(xpos_list)))  
    mj_np = mj_np[(mj_np[:, 0] > 0.4) & (mj_np[:, 0] < 0.6)] 
    time_b = mj_np[:, 0]  
    values_b = mj_np[:, 1]  

    # 使用 np.interp 进行插值，将 b 的值插值到 a 的时间点  
    interpolated_values_b = np.interp(time_a, time_b, values_b)  
    absolute_difference = np.abs(values_a - interpolated_values_b)  
    sum_difference = np.sum(absolute_difference, axis=0) 
    return sum_difference

In [ ]:
loss_function(d0=0.01, d_width=0.01)

In [ ]:
0.306/12

In [ ]:
0.271/24

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mujoco as mj
import time
import pandas as pd

In [ ]:
xlsx_path = './tensegrity_jump0828.xlsx'
df = pd.read_excel(xlsx_path, sheet_name='Sheet1')

In [ ]:
# 绘制df的图像
plt.plot(df['time'], df['angvel'])
plt.xlabel('time (s)')
plt.ylabel('angular velocity (rad/s)')
plt.grid()
plt.show()

In [ ]:
plt.plot(df['time'], df['h'])
plt.xlabel('time (s)')
plt.ylabel('height (mm)')
plt.grid()
plt.show()

In [ ]:
adams_df = df.copy()
adams_df['h'] = adams_df['h'] * 0.001
adams_np = adams_df.to_numpy()

In [ ]:
from concurrent import futures
import nevergrad as ng
import matplotlib.pyplot as plt
import numpy as np
import mujoco as mj
import time
import pandas as pd
import xml.etree.ElementTree as ET


xml_path = '../data/xml/TT12_0828.xml'
xlsx_path = './tensegrity_jump0828.xlsx'

df = pd.read_excel(xlsx_path, sheet_name='Sheet1')
adams_df = df.copy()
adams_df['h'] = adams_df['h'] * 0.001
adams_np = adams_df.to_numpy()

def update_floor_solref_solimp(xml_file, solref, solimp):
    tree = ET.parse(xml_file)
    worldbody = tree.find('worldbody')
    geoms = worldbody.findall('geom')
    for geom in geoms:
        if geom.get('name') == 'floor':
            geom.set('solref', solref)
            geom.set('solimp', solimp)
    return ET.tostring(tree.getroot(), encoding='unicode')

def controller(model, data):
    torque_x_actuator_id = mj.mj_name2id(model, mj.mjtObj.mjOBJ_ACTUATOR, "tt_model_0/torque_x_ctrl")

    if abs(data.sensor('tt_model_0/angvel_inside_ball').data[0] - data.sensor('tt_model_0/angvel_outside_ball').data[0]) < 0.1:
        data.ctrl[torque_x_actuator_id] = 0

def simulate_step(model, data, Hz):
    current_simstart = data.time   # 当前一帧的开始时间
    while (data.time - current_simstart < 1.0/Hz):
        mj.mj_step(model, data)

def loss_function(d0=0.27, d_width=0.07, width=0.06, stiffness=5773, damping=68):
    xml_string =update_floor_solref_solimp(xml_path, solref=f'{-stiffness} {-damping}', solimp=f'{d0} {d_width} {width} 0.5 2')
    
    m = mj.MjModel.from_xml_string(xml_string)
    d = mj.MjData(m)

    mj.mj_resetDataKeyframe(m, d, 0)
    mj.mj_forward(m, d)
    mj.set_mjcb_control(controller)
    Hz = 1000
    time_list = []
    outside_z_pos_list = []
    angvel_outside_ball_list = []

    while d.time < 1.5:
        simulate_step(m, d, Hz)
        time_list.append(d.time)
        outside_z_pos_list.append(d.sensor('tt_model_0/pos_outside_ball').data[2])
        angvel_outside_ball_list.append(d.sensor('tt_model_0/angvel_outside_ball').data[0])
    
    mj_np = np.column_stack((np.array(time_list), np.array(angvel_outside_ball_list), np.array(outside_z_pos_list)))
    interpolated_values_angvel = np.interp(adams_np[:,0], mj_np[:,0], mj_np[:,1])
    interpolated_values_height = np.interp(adams_np[:,0], mj_np[:,0], mj_np[:,2])
    absolute_difference = np.abs(adams_np[:, 1] - interpolated_values_angvel) + np.abs(adams_np[:, 2] - interpolated_values_height)
    sum_difference = np.sum(absolute_difference, axis=0)
    del m
    del d
    return sum_difference

In [2]:
import mujoco as mj
import numpy as np

d0=0.27
d_width=0.07
width=0.06
stiffness=5773
damping=68

friction1 = 0.6
friction2 = 0.005
friction3 = 0.0001

a0 = 10
Hz = 1000
BLOCK_HALF_WIDTH = 0.05
mass = 1

pos = np.array([0, 0, 0.05])
key1pos = np.array([0, 0, 1])
quat = np.array([0, 0, 0, 1])
vel = np.array([0, 0, 0])
angvel = np.array([0, 0, 0])

xml_string=f"""
        <mujoco>
        <option integrator="implicit" 
            o_solref="{-stiffness} {-damping}" 
            o_solimp="{d0} {d_width} {width} 0.5 1" 
            o_friction = '{friction1} {friction2} {friction3}'
            timestep="{1/Hz}" 
            gravity="0 0 -{a0}" >
        <flag override="disable" />
        </option>
        <worldbody>
            <light name="light_-2"  pos="-2 -1 3" dir="0.666667 0.333333 -0.666667"/>
            <light name="light_2"  pos="2 -1 3" dir="-0.666667 0.333333 -0.666667"/>
            <geom name='floor' 
                type='plane' 
                size='0 0 0.01' 
                priority='1' 
                solref="{-stiffness} {-damping}"
                solimp="{d0} {d_width} {width} 0.5 2" 
                friction='{friction1} {friction2} {friction3}'/>
            <body name='box' pos="{pos[0]} {pos[1]} {pos[2]}" quat='{quat[0]} {quat[1]} {quat[2]} {quat[3]}'>
                <freejoint/>
                <inertial pos='0 0 0' mass='0.37' diaginertia='0.00081 0.00081 0.00081'/>
                <geom name='box' 
                    type='box' 
                    size="{BLOCK_HALF_WIDTH} {BLOCK_HALF_WIDTH} {BLOCK_HALF_WIDTH}" 
                    rgba="0.8 0.1 0.1 1"
                    mass="{mass}"/>
            </body>
        </worldbody>
        <keyframe>
            <key name='initial_state' qvel='{vel[0]} {vel[1]} {vel[2]} {angvel[0]} {angvel[1]} {angvel[2]}'/>
            <key name='initial_state1' qpos='{key1pos[0]} {key1pos[1]} {key1pos[2]} 0 0 0 1' qvel='{vel[0]} {vel[1]} {vel[2]} {angvel[0]} {angvel[1]} {angvel[2]}'/>
        </keyframe>
        </mujoco>
        """

mj.MjModel.from_xml_string(xml_string)

In [5]:
m = mj.MjModel.from_xml_string(xml_string)

In [8]:
m = mj.MjModel.from_xml_path("../data/xml/TT12_0828.xml")

In [19]:
xml_string = """
<?xml version='1.0' encoding='utf-8'?>
<mujoco model="TT12_environment">
  <compiler autolimits="true" coordinate="local" angle="degree" inertiafromgeom="true" />
  <option timestep="0.00001" gravity="0 0 -10" integrator="Euler" cone="elliptic">
    <flag override="disable" energy="enable" />
  </option>
  <visual>
    <headlight ambient="0.59999999999999998 0.59999999999999998 0.59999999999999998" diffuse="0.29999999999999999 0.29999999999999999 0.29999999999999999" specular="0.10000000000000001 0.10000000000000001 0.10000000000000001" />
  </visual>
  <default>
    <default class="/">
      <joint type="hinge" damping="0" />
      <geom type="capsule" />
      <site type="sphere" size="0.01" />
      <tendon limited="true" range="0 10" width="0.0050000000000000001" rgba="1 0 0 1" damping="10" />
    </default>
    <default class="tt_model_0/">
      <tendon limited="true" range="0 10" width="0.0050000000000000001" rgba="1 0 0 1" damping="10" />
    </default>
    <default class="tt_model_0/rod0_1/" />
    <default class="tt_model_0/rod2_3/" />
    <default class="tt_model_0/rod4_5/" />
    <default class="tt_model_0/rod6_7/" />
    <default class="tt_model_0/rod8_9/" />
    <default class="tt_model_0/rod10_11/" />
    <default class="tt_model_0/rod12_13/" />
    <default class="tt_model_0/rod14_15/" />
    <default class="tt_model_0/rod16_17/" />
    <default class="tt_model_0/rod18_19/" />
    <default class="tt_model_0/rod20_21/" />
    <default class="tt_model_0/rod22_23/" />
    <default class="tt_model_0/middle_platform/" />
  </default>
  <asset>
    <texture name="skybox" type="skybox" builtin="gradient" rgb1="0.29999999999999999 0.5 0.69999999999999996" rgb2="0 0 0" width="512" height="3072" />
    <texture name="2d" type="2d" builtin="checker" rgb1="0.20000000000000001 0.29999999999999999 0.40000000000000002" rgb2="0.10000000000000001 0.20000000000000001 0.29999999999999999" width="300" height="300" />
    <material name="grid" class="/" texture="2d" texrepeat="5 5" reflectance="0.20000000000000001" />
  </asset>
  <worldbody>
    <geom name="floor" class="/" type="plane" priority="1" size="0 0 0.125" material="grid" friction="0.5 0.0050000000000000001 0.0001" solref="-5773.1000000000004 -68.760000000000005" solimp="0.26900000000000002 0.070999999999999994 0.065000000000000002 0.5 2" pos="0 0 0" />
    <light name="light_-2" class="/" pos="-2 -1 3" dir="2 1 -2" />
    <light name="light_2" class="/" pos="2 -1 3" dir="-2 1 -2" />
    <site name="world_site" class="/" pos="0 0 0" />
    <site name="tt_model_0/middle_site_0_1" class="tt_model_0/" rgba="0 0 0 0" pos="0.17499999999999999 0.17499999999999999 1" />
    <body pos="0.17499999999999999 0.17499999999999999 1" name="tt_model_0/rod0_1/">
      <freejoint name="tt_model_0/rod0_1/" />
      <geom name="tt_model_0/rod0_1/geom0_1" class="tt_model_0/rod0_1/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0 0.35000000000000009 0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod0_1/node_0" class="tt_model_0/rod0_1/" pos="0 0 0.35000000000000009" />
      <site name="tt_model_0/rod0_1/node_1" class="tt_model_0/rod0_1/" pos="0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod0_1/node_24" class="tt_model_0/rod0_1/" pos="0 0 0" />
      <geom name="tt_model_0/rod0_1/geom_edge_0" class="tt_model_0/rod0_1/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 0.35000000000000009" />
      <geom name="tt_model_0/rod0_1/geom_edge_1" class="tt_model_0/rod0_1/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 -0.34999999999999998" />
    </body>
    <site name="tt_model_0/middle_site_2_3" class="tt_model_0/" rgba="0 0 0 0" pos="-0.17499999999999999 0.17499999999999999 1" />
    <body pos="-0.17499999999999999 0.17499999999999999 1" name="tt_model_0/rod2_3/">
      <freejoint name="tt_model_0/rod2_3/" />
      <geom name="tt_model_0/rod2_3/geom2_3" class="tt_model_0/rod2_3/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0 0.35000000000000009 0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod2_3/node_2" class="tt_model_0/rod2_3/" pos="0 0 0.35000000000000009" />
      <site name="tt_model_0/rod2_3/node_3" class="tt_model_0/rod2_3/" pos="0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod2_3/node_25" class="tt_model_0/rod2_3/" pos="0 0 0" />
      <geom name="tt_model_0/rod2_3/geom_edge_2" class="tt_model_0/rod2_3/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 0.35000000000000009" />
      <geom name="tt_model_0/rod2_3/geom_edge_3" class="tt_model_0/rod2_3/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 -0.34999999999999998" />
    </body>
    <site name="tt_model_0/middle_site_4_5" class="tt_model_0/" rgba="0 0 0 0" pos="-0.17499999999999999 -0.17499999999999999 1" />
    <body pos="-0.17499999999999999 -0.17499999999999999 1" name="tt_model_0/rod4_5/">
      <freejoint name="tt_model_0/rod4_5/" />
      <geom name="tt_model_0/rod4_5/geom4_5" class="tt_model_0/rod4_5/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0 0.35000000000000009 0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod4_5/node_4" class="tt_model_0/rod4_5/" pos="0 0 0.35000000000000009" />
      <site name="tt_model_0/rod4_5/node_5" class="tt_model_0/rod4_5/" pos="0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod4_5/node_26" class="tt_model_0/rod4_5/" pos="0 0 0" />
      <geom name="tt_model_0/rod4_5/geom_edge_4" class="tt_model_0/rod4_5/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 0.35000000000000009" />
      <geom name="tt_model_0/rod4_5/geom_edge_5" class="tt_model_0/rod4_5/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 -0.34999999999999998" />
    </body>
    <site name="tt_model_0/middle_site_6_7" class="tt_model_0/" rgba="0 0 0 0" pos="0.17499999999999999 -0.17499999999999999 1" />
    <body pos="0.17499999999999999 -0.17499999999999999 1" name="tt_model_0/rod6_7/">
      <freejoint name="tt_model_0/rod6_7/" />
      <geom name="tt_model_0/rod6_7/geom6_7" class="tt_model_0/rod6_7/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0 0.35000000000000009 0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod6_7/node_6" class="tt_model_0/rod6_7/" pos="0 0 0.35000000000000009" />
      <site name="tt_model_0/rod6_7/node_7" class="tt_model_0/rod6_7/" pos="0 0 -0.34999999999999998" />
      <site name="tt_model_0/rod6_7/node_27" class="tt_model_0/rod6_7/" pos="0 0 0" />
      <geom name="tt_model_0/rod6_7/geom_edge_6" class="tt_model_0/rod6_7/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 0.35000000000000009" />
      <geom name="tt_model_0/rod6_7/geom_edge_7" class="tt_model_0/rod6_7/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0 -0.34999999999999998" />
    </body>
    <site name="tt_model_0/middle_site_8_9" class="tt_model_0/" rgba="0 0 0 0" pos="0.17499999999999999 0 1.175" />
    <body pos="0.17499999999999999 0 1.175" name="tt_model_0/rod8_9/">
      <freejoint name="tt_model_0/rod8_9/" />
      <geom name="tt_model_0/rod8_9/geom8_9" class="tt_model_0/rod8_9/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0.34999999999999998 0 0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod8_9/node_8" class="tt_model_0/rod8_9/" pos="0 0.34999999999999998 0" />
      <site name="tt_model_0/rod8_9/node_9" class="tt_model_0/rod8_9/" pos="0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod8_9/node_28" class="tt_model_0/rod8_9/" pos="0 0 0" />
      <geom name="tt_model_0/rod8_9/geom_edge_8" class="tt_model_0/rod8_9/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0.34999999999999998 0" />
      <geom name="tt_model_0/rod8_9/geom_edge_9" class="tt_model_0/rod8_9/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 -0.34999999999999998 0" />
    </body>
    <site name="tt_model_0/middle_site_10_11" class="tt_model_0/" rgba="0 0 0 0" pos="0.17499999999999999 0 0.82499999999999996" />
    <body pos="0.17499999999999999 0 0.82499999999999996" name="tt_model_0/rod10_11/">
      <freejoint name="tt_model_0/rod10_11/" />
      <geom name="tt_model_0/rod10_11/geom10_11" class="tt_model_0/rod10_11/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0.34999999999999998 0 0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod10_11/node_10" class="tt_model_0/rod10_11/" pos="0 0.34999999999999998 0" />
      <site name="tt_model_0/rod10_11/node_11" class="tt_model_0/rod10_11/" pos="0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod10_11/node_29" class="tt_model_0/rod10_11/" pos="0 0 0" />
      <geom name="tt_model_0/rod10_11/geom_edge_10" class="tt_model_0/rod10_11/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0.34999999999999998 0" />
      <geom name="tt_model_0/rod10_11/geom_edge_11" class="tt_model_0/rod10_11/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 -0.34999999999999998 0" />
    </body>
    <site name="tt_model_0/middle_site_12_13" class="tt_model_0/" rgba="0 0 0 0" pos="-0.17499999999999999 0 0.82499999999999996" />
    <body pos="-0.17499999999999999 0 0.82499999999999996" name="tt_model_0/rod12_13/">
      <freejoint name="tt_model_0/rod12_13/" />
      <geom name="tt_model_0/rod12_13/geom12_13" class="tt_model_0/rod12_13/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0.34999999999999998 0 0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod12_13/node_12" class="tt_model_0/rod12_13/" pos="0 0.34999999999999998 0" />
      <site name="tt_model_0/rod12_13/node_13" class="tt_model_0/rod12_13/" pos="0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod12_13/node_30" class="tt_model_0/rod12_13/" pos="0 0 0" />
      <geom name="tt_model_0/rod12_13/geom_edge_12" class="tt_model_0/rod12_13/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0.34999999999999998 0" />
      <geom name="tt_model_0/rod12_13/geom_edge_13" class="tt_model_0/rod12_13/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 -0.34999999999999998 0" />
    </body>
    <site name="tt_model_0/middle_site_14_15" class="tt_model_0/" rgba="0 0 0 0" pos="-0.17499999999999999 0 1.175" />
    <body pos="-0.17499999999999999 0 1.175" name="tt_model_0/rod14_15/">
      <freejoint name="tt_model_0/rod14_15/" />
      <geom name="tt_model_0/rod14_15/geom14_15" class="tt_model_0/rod14_15/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0 0.34999999999999998 0 0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod14_15/node_14" class="tt_model_0/rod14_15/" pos="0 0.34999999999999998 0" />
      <site name="tt_model_0/rod14_15/node_15" class="tt_model_0/rod14_15/" pos="0 -0.34999999999999998 0" />
      <site name="tt_model_0/rod14_15/node_31" class="tt_model_0/rod14_15/" pos="0 0 0" />
      <geom name="tt_model_0/rod14_15/geom_edge_14" class="tt_model_0/rod14_15/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 0.34999999999999998 0" />
      <geom name="tt_model_0/rod14_15/geom_edge_15" class="tt_model_0/rod14_15/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0 -0.34999999999999998 0" />
    </body>
    <site name="tt_model_0/middle_site_16_17" class="tt_model_0/" rgba="0 0 0 0" pos="0 0.17499999999999999 1.175" />
    <body pos="0 0.17499999999999999 1.175" name="tt_model_0/rod16_17/">
      <freejoint name="tt_model_0/rod16_17/" />
      <geom name="tt_model_0/rod16_17/geom16_17" class="tt_model_0/rod16_17/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0.34999999999999998 0 0 -0.34999999999999998 0 0" />
      <site name="tt_model_0/rod16_17/node_16" class="tt_model_0/rod16_17/" pos="0.34999999999999998 0 0" />
      <site name="tt_model_0/rod16_17/node_17" class="tt_model_0/rod16_17/" pos="-0.34999999999999998 0 0" />
      <site name="tt_model_0/rod16_17/node_32" class="tt_model_0/rod16_17/" pos="0 0 0" />
      <geom name="tt_model_0/rod16_17/geom_edge_16" class="tt_model_0/rod16_17/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0.34999999999999998 0 0" />
      <geom name="tt_model_0/rod16_17/geom_edge_17" class="tt_model_0/rod16_17/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="-0.34999999999999998 0 0" />
    </body>
    <site name="tt_model_0/middle_site_18_19" class="tt_model_0/" rgba="0 0 0 0" pos="0 -0.17499999999999999 1.175" />
    <body pos="0 -0.17499999999999999 1.175" name="tt_model_0/rod18_19/">
      <freejoint name="tt_model_0/rod18_19/" />
      <geom name="tt_model_0/rod18_19/geom18_19" class="tt_model_0/rod18_19/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0.34999999999999998 0 0 -0.34999999999999998 0 0" />
      <site name="tt_model_0/rod18_19/node_18" class="tt_model_0/rod18_19/" pos="0.34999999999999998 0 0" />
      <site name="tt_model_0/rod18_19/node_19" class="tt_model_0/rod18_19/" pos="-0.34999999999999998 0 0" />
      <site name="tt_model_0/rod18_19/node_33" class="tt_model_0/rod18_19/" pos="0 0 0" />
      <geom name="tt_model_0/rod18_19/geom_edge_18" class="tt_model_0/rod18_19/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0.34999999999999998 0 0" />
      <geom name="tt_model_0/rod18_19/geom_edge_19" class="tt_model_0/rod18_19/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="-0.34999999999999998 0 0" />
    </body>
    <site name="tt_model_0/middle_site_20_21" class="tt_model_0/" rgba="0 0 0 0" pos="0 -0.17499999999999999 0.82499999999999996" />
    <body pos="0 -0.17499999999999999 0.82499999999999996" name="tt_model_0/rod20_21/">
      <freejoint name="tt_model_0/rod20_21/" />
      <geom name="tt_model_0/rod20_21/geom20_21" class="tt_model_0/rod20_21/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0.34999999999999998 0 0 -0.34999999999999998 0 0" />
      <site name="tt_model_0/rod20_21/node_20" class="tt_model_0/rod20_21/" pos="0.34999999999999998 0 0" />
      <site name="tt_model_0/rod20_21/node_21" class="tt_model_0/rod20_21/" pos="-0.34999999999999998 0 0" />
      <site name="tt_model_0/rod20_21/node_34" class="tt_model_0/rod20_21/" pos="0 0 0" />
      <geom name="tt_model_0/rod20_21/geom_edge_20" class="tt_model_0/rod20_21/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0.34999999999999998 0 0" />
      <geom name="tt_model_0/rod20_21/geom_edge_21" class="tt_model_0/rod20_21/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="-0.34999999999999998 0 0" />
    </body>
    <site name="tt_model_0/middle_site_22_23" class="tt_model_0/" rgba="0 0 0 0" pos="0 0.17499999999999999 0.82499999999999996" />
    <body pos="0 0.17499999999999999 0.82499999999999996" name="tt_model_0/rod22_23/">
      <freejoint name="tt_model_0/rod22_23/" />
      <geom name="tt_model_0/rod22_23/geom22_23" class="tt_model_0/rod22_23/" type="cylinder" size="0.012" mass="0.022499999999999999" fromto="0.34999999999999998 0 0 -0.34999999999999998 0 0" />
      <site name="tt_model_0/rod22_23/node_22" class="tt_model_0/rod22_23/" pos="0.34999999999999998 0 0" />
      <site name="tt_model_0/rod22_23/node_23" class="tt_model_0/rod22_23/" pos="-0.34999999999999998 0 0" />
      <site name="tt_model_0/rod22_23/node_35" class="tt_model_0/rod22_23/" pos="0 0 0" />
      <geom name="tt_model_0/rod22_23/geom_edge_22" class="tt_model_0/rod22_23/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="0.34999999999999998 0 0" />
      <geom name="tt_model_0/rod22_23/geom_edge_23" class="tt_model_0/rod22_23/" type="sphere" size="0.029999999999999999" rgba="0 0.90000000000000002 0 0.5" mass="0.011299999999999999" pos="-0.34999999999999998 0 0" />
    </body>
    <site name="tt_model_0/middle_element_site" class="tt_model_0/" rgba="0 0 0 0" pos="0 0 1" />
    <body pos="0 0 1" name="tt_model_0/middle_platform/">
      <freejoint name="tt_model_0/middle_platform/" />
      <inertial pos="0 0 0" mass="0.69999999999999996" diaginertia="0.02 0.02 0.02" />
      <site name="tt_model_0/middle_platform/node_40" class="tt_model_0/middle_platform/" pos="0 -0.34999999999999998 0" />
      <site name="tt_model_0/middle_platform/node_38" class="tt_model_0/middle_platform/" pos="-0.34999999999999998 0 0" />
      <site name="tt_model_0/middle_platform/node_42" class="tt_model_0/middle_platform/" pos="0 0 -0.34999999999999998" />
      <site name="tt_model_0/middle_platform/node_41" class="tt_model_0/middle_platform/" pos="0 0 0.35000000000000009" />
      <site name="tt_model_0/middle_platform/node_37" class="tt_model_0/middle_platform/" pos="0.34999999999999998 0 0" />
      <site name="tt_model_0/middle_platform/node_39" class="tt_model_0/middle_platform/" pos="0 0.34999999999999998 0" />
      <site name="tt_model_0/middle_platform/outside_site" class="tt_model_0/middle_platform/" rgba="1 0.90000000000000002 0 0.5" size="0.15 0.15 0.15" type='box' pos="0 0 0" />
      <body name="tt_model_0/middle_platform/inside_ball" pos="0 0 0">
        <inertial pos="0 0 0" mass="6.5999999999999996" diaginertia="0.12 0.12 0.12" />
        <joint name="tt_model_0/middle_platform/x_control" class="tt_model_0/middle_platform/" type="hinge" axis="1 0 0" />
        <joint name="tt_model_0/middle_platform/y_control" class="tt_model_0/middle_platform/" type="hinge" axis="0 1 0" />
        <joint name="tt_model_0/middle_platform/z_control" class="tt_model_0/middle_platform/" type="hinge" axis="0 0 1" />
        <site name="tt_model_0/middle_platform/inside_site" class="tt_model_0/middle_platform/" type="box" rgba="0 0.90000000000000002 0 1" size="0.10000000000000001 0.10000000000000001 0.10000000000000001" pos="0 0 0" />
      </body>
    </body>
  </worldbody>
  
  <contact>
    <exclude name="tt_model_0/exclude_rod0_1_rod2_3" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod2_3/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod4_5" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod4_5/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod6_7" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod6_7/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod8_9" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod8_9/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod10_11" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod10_11/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod12_13" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod12_13/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod14_15" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod16_17" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod18_19" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod20_21" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod0_1_rod22_23" body1="tt_model_0/rod0_1/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod4_5" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod4_5/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod6_7" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod6_7/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod8_9" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod8_9/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod10_11" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod10_11/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod12_13" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod12_13/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod14_15" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod16_17" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod18_19" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod20_21" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod2_3_rod22_23" body1="tt_model_0/rod2_3/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod6_7" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod6_7/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod8_9" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod8_9/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod10_11" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod10_11/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod12_13" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod12_13/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod14_15" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod16_17" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod18_19" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod20_21" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod4_5_rod22_23" body1="tt_model_0/rod4_5/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod8_9" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod8_9/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod10_11" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod10_11/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod12_13" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod12_13/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod14_15" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod16_17" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod18_19" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod20_21" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod6_7_rod22_23" body1="tt_model_0/rod6_7/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod10_11" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod10_11/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod12_13" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod12_13/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod14_15" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod16_17" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod18_19" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod20_21" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod8_9_rod22_23" body1="tt_model_0/rod8_9/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod10_11_rod12_13" body1="tt_model_0/rod10_11/" body2="tt_model_0/rod12_13/" />
    <exclude name="tt_model_0/exclude_rod10_11_rod14_15" body1="tt_model_0/rod10_11/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod10_11_rod16_17" body1="tt_model_0/rod10_11/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod10_11_rod18_19" body1="tt_model_0/rod10_11/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod10_11_rod20_21" body1="tt_model_0/rod10_11/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod10_11_rod22_23" body1="tt_model_0/rod10_11/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod12_13_rod14_15" body1="tt_model_0/rod12_13/" body2="tt_model_0/rod14_15/" />
    <exclude name="tt_model_0/exclude_rod12_13_rod16_17" body1="tt_model_0/rod12_13/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod12_13_rod18_19" body1="tt_model_0/rod12_13/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod12_13_rod20_21" body1="tt_model_0/rod12_13/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod12_13_rod22_23" body1="tt_model_0/rod12_13/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod14_15_rod16_17" body1="tt_model_0/rod14_15/" body2="tt_model_0/rod16_17/" />
    <exclude name="tt_model_0/exclude_rod14_15_rod18_19" body1="tt_model_0/rod14_15/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod14_15_rod20_21" body1="tt_model_0/rod14_15/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod14_15_rod22_23" body1="tt_model_0/rod14_15/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod16_17_rod18_19" body1="tt_model_0/rod16_17/" body2="tt_model_0/rod18_19/" />
    <exclude name="tt_model_0/exclude_rod16_17_rod20_21" body1="tt_model_0/rod16_17/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod16_17_rod22_23" body1="tt_model_0/rod16_17/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod18_19_rod20_21" body1="tt_model_0/rod18_19/" body2="tt_model_0/rod20_21/" />
    <exclude name="tt_model_0/exclude_rod18_19_rod22_23" body1="tt_model_0/rod18_19/" body2="tt_model_0/rod22_23/" />
    <exclude name="tt_model_0/exclude_rod20_21_rod22_23" body1="tt_model_0/rod20_21/" body2="tt_model_0/rod22_23/" />
  </contact>
  <tendon>
    <spatial name="tt_model_0/td0_2" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod0_1/node_0" />
      <site site="tt_model_0/rod2_3/node_2" />
    </spatial>
    <spatial name="tt_model_0/td2_4" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod2_3/node_2" />
      <site site="tt_model_0/rod4_5/node_4" />
    </spatial>
    <spatial name="tt_model_0/td4_6" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod4_5/node_4" />
      <site site="tt_model_0/rod6_7/node_6" />
    </spatial>
    <spatial name="tt_model_0/td6_0" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod6_7/node_6" />
      <site site="tt_model_0/rod0_1/node_0" />
    </spatial>
    <spatial name="tt_model_0/td1_3" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod0_1/node_1" />
      <site site="tt_model_0/rod2_3/node_3" />
    </spatial>
    <spatial name="tt_model_0/td3_5" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod2_3/node_3" />
      <site site="tt_model_0/rod4_5/node_5" />
    </spatial>
    <spatial name="tt_model_0/td5_7" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod4_5/node_5" />
      <site site="tt_model_0/rod6_7/node_7" />
    </spatial>
    <spatial name="tt_model_0/td7_1" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod6_7/node_7" />
      <site site="tt_model_0/rod0_1/node_1" />
    </spatial>
    <spatial name="tt_model_0/td8_10" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod8_9/node_8" />
      <site site="tt_model_0/rod10_11/node_10" />
    </spatial>
    <spatial name="tt_model_0/td10_12" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod10_11/node_10" />
      <site site="tt_model_0/rod12_13/node_12" />
    </spatial>
    <spatial name="tt_model_0/td12_14" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod12_13/node_12" />
      <site site="tt_model_0/rod14_15/node_14" />
    </spatial>
    <spatial name="tt_model_0/td14_8" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod14_15/node_14" />
      <site site="tt_model_0/rod8_9/node_8" />
    </spatial>
    <spatial name="tt_model_0/td9_11" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod8_9/node_9" />
      <site site="tt_model_0/rod10_11/node_11" />
    </spatial>
    <spatial name="tt_model_0/td11_13" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod10_11/node_11" />
      <site site="tt_model_0/rod12_13/node_13" />
    </spatial>
    <spatial name="tt_model_0/td13_15" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod12_13/node_13" />
      <site site="tt_model_0/rod14_15/node_15" />
    </spatial>
    <spatial name="tt_model_0/td15_9" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod14_15/node_15" />
      <site site="tt_model_0/rod8_9/node_9" />
    </spatial>
    <spatial name="tt_model_0/td16_18" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod16_17/node_16" />
      <site site="tt_model_0/rod18_19/node_18" />
    </spatial>
    <spatial name="tt_model_0/td18_20" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod18_19/node_18" />
      <site site="tt_model_0/rod20_21/node_20" />
    </spatial>
    <spatial name="tt_model_0/td20_22" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod20_21/node_20" />
      <site site="tt_model_0/rod22_23/node_22" />
    </spatial>
    <spatial name="tt_model_0/td22_16" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod22_23/node_22" />
      <site site="tt_model_0/rod16_17/node_16" />
    </spatial>
    <spatial name="tt_model_0/td17_19" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod16_17/node_17" />
      <site site="tt_model_0/rod18_19/node_19" />
    </spatial>
    <spatial name="tt_model_0/td19_21" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod18_19/node_19" />
      <site site="tt_model_0/rod20_21/node_21" />
    </spatial>
    <spatial name="tt_model_0/td21_23" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod20_21/node_21" />
      <site site="tt_model_0/rod22_23/node_23" />
    </spatial>
    <spatial name="tt_model_0/td23_17" class="tt_model_0/" rgba="1 0.5 0.5 0.69999999999999996" springlength="0.22" stiffness="771">
      <site site="tt_model_0/rod22_23/node_23" />
      <site site="tt_model_0/rod16_17/node_17" />
    </spatial>
    <spatial name="tt_model_0/td0_16" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod0_1/node_0" />
      <site site="tt_model_0/rod16_17/node_16" />
    </spatial>
    <spatial name="tt_model_0/td2_17" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod2_3/node_2" />
      <site site="tt_model_0/rod16_17/node_17" />
    </spatial>
    <spatial name="tt_model_0/td4_19" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod4_5/node_4" />
      <site site="tt_model_0/rod18_19/node_19" />
    </spatial>
    <spatial name="tt_model_0/td6_18" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod6_7/node_6" />
      <site site="tt_model_0/rod18_19/node_18" />
    </spatial>
    <spatial name="tt_model_0/td1_22" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod0_1/node_1" />
      <site site="tt_model_0/rod22_23/node_22" />
    </spatial>
    <spatial name="tt_model_0/td3_23" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod2_3/node_3" />
      <site site="tt_model_0/rod22_23/node_23" />
    </spatial>
    <spatial name="tt_model_0/td5_21" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod4_5/node_5" />
      <site site="tt_model_0/rod20_21/node_21" />
    </spatial>
    <spatial name="tt_model_0/td7_20" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod6_7/node_7" />
      <site site="tt_model_0/rod20_21/node_20" />
    </spatial>
    <spatial name="tt_model_0/td8_0" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod8_9/node_8" />
      <site site="tt_model_0/rod0_1/node_0" />
    </spatial>
    <spatial name="tt_model_0/td8_16" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod8_9/node_8" />
      <site site="tt_model_0/rod16_17/node_16" />
    </spatial>
    <spatial name="tt_model_0/td9_6" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod8_9/node_9" />
      <site site="tt_model_0/rod6_7/node_6" />
    </spatial>
    <spatial name="tt_model_0/td9_18" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod8_9/node_9" />
      <site site="tt_model_0/rod18_19/node_18" />
    </spatial>
    <spatial name="tt_model_0/td10_1" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod10_11/node_10" />
      <site site="tt_model_0/rod0_1/node_1" />
    </spatial>
    <spatial name="tt_model_0/td10_22" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod10_11/node_10" />
      <site site="tt_model_0/rod22_23/node_22" />
    </spatial>
    <spatial name="tt_model_0/td11_7" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod10_11/node_11" />
      <site site="tt_model_0/rod6_7/node_7" />
    </spatial>
    <spatial name="tt_model_0/td11_20" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod10_11/node_11" />
      <site site="tt_model_0/rod20_21/node_20" />
    </spatial>
    <spatial name="tt_model_0/td12_3" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod12_13/node_12" />
      <site site="tt_model_0/rod2_3/node_3" />
    </spatial>
    <spatial name="tt_model_0/td12_23" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod12_13/node_12" />
      <site site="tt_model_0/rod22_23/node_23" />
    </spatial>
    <spatial name="tt_model_0/td13_5" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod12_13/node_13" />
      <site site="tt_model_0/rod4_5/node_5" />
    </spatial>
    <spatial name="tt_model_0/td13_21" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod12_13/node_13" />
      <site site="tt_model_0/rod20_21/node_21" />
    </spatial>
    <spatial name="tt_model_0/td14_2" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod14_15/node_14" />
      <site site="tt_model_0/rod2_3/node_2" />
    </spatial>
    <spatial name="tt_model_0/td14_17" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod14_15/node_14" />
      <site site="tt_model_0/rod16_17/node_17" />
    </spatial>
    <spatial name="tt_model_0/td15_4" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod14_15/node_15" />
      <site site="tt_model_0/rod4_5/node_4" />
    </spatial>
    <spatial name="tt_model_0/td15_19" class="tt_model_0/" rgba="0.69999999999999996 0 0 0.69999999999999996" springlength="0.10000000000000001" stiffness="2000">
      <site site="tt_model_0/rod14_15/node_15" />
      <site site="tt_model_0/rod18_19/node_19" />
    </spatial>
    <spatial name="tt_model_0/td24_37" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod0_1/node_24" />
      <site site="tt_model_0/middle_platform/node_37" />
    </spatial>
    <spatial name="tt_model_0/td28_37" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod8_9/node_28" />
      <site site="tt_model_0/middle_platform/node_37" />
    </spatial>
    <spatial name="tt_model_0/td27_37" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod6_7/node_27" />
      <site site="tt_model_0/middle_platform/node_37" />
    </spatial>
    <spatial name="tt_model_0/td29_37" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod10_11/node_29" />
      <site site="tt_model_0/middle_platform/node_37" />
    </spatial>
    <spatial name="tt_model_0/td25_38" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod2_3/node_25" />
      <site site="tt_model_0/middle_platform/node_38" />
    </spatial>
    <spatial name="tt_model_0/td31_38" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod14_15/node_31" />
      <site site="tt_model_0/middle_platform/node_38" />
    </spatial>
    <spatial name="tt_model_0/td26_38" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod4_5/node_26" />
      <site site="tt_model_0/middle_platform/node_38" />
    </spatial>
    <spatial name="tt_model_0/td30_38" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod12_13/node_30" />
      <site site="tt_model_0/middle_platform/node_38" />
    </spatial>
    <spatial name="tt_model_0/td35_39" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod22_23/node_35" />
      <site site="tt_model_0/middle_platform/node_39" />
    </spatial>
    <spatial name="tt_model_0/td25_39" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod2_3/node_25" />
      <site site="tt_model_0/middle_platform/node_39" />
    </spatial>
    <spatial name="tt_model_0/td32_39" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod16_17/node_32" />
      <site site="tt_model_0/middle_platform/node_39" />
    </spatial>
    <spatial name="tt_model_0/td24_39" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod0_1/node_24" />
      <site site="tt_model_0/middle_platform/node_39" />
    </spatial>
    <spatial name="tt_model_0/td34_40" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod20_21/node_34" />
      <site site="tt_model_0/middle_platform/node_40" />
    </spatial>
    <spatial name="tt_model_0/td26_40" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod4_5/node_26" />
      <site site="tt_model_0/middle_platform/node_40" />
    </spatial>
    <spatial name="tt_model_0/td33_40" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod18_19/node_33" />
      <site site="tt_model_0/middle_platform/node_40" />
    </spatial>
    <spatial name="tt_model_0/td27_40" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod6_7/node_27" />
      <site site="tt_model_0/middle_platform/node_40" />
    </spatial>
    <spatial name="tt_model_0/td32_41" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod16_17/node_32" />
      <site site="tt_model_0/middle_platform/node_41" />
    </spatial>
    <spatial name="tt_model_0/td31_41" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod14_15/node_31" />
      <site site="tt_model_0/middle_platform/node_41" />
    </spatial>
    <spatial name="tt_model_0/td33_41" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod18_19/node_33" />
      <site site="tt_model_0/middle_platform/node_41" />
    </spatial>
    <spatial name="tt_model_0/td28_41" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod8_9/node_28" />
      <site site="tt_model_0/middle_platform/node_41" />
    </spatial>
    <spatial name="tt_model_0/td35_42" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod22_23/node_35" />
      <site site="tt_model_0/middle_platform/node_42" />
    </spatial>
    <spatial name="tt_model_0/td30_42" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod12_13/node_30" />
      <site site="tt_model_0/middle_platform/node_42" />
    </spatial>
    <spatial name="tt_model_0/td29_42" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod10_11/node_29" />
      <site site="tt_model_0/middle_platform/node_42" />
    </spatial>
    <spatial name="tt_model_0/td34_42" class="tt_model_0/" rgba="1 1 0 0.69999999999999996" springlength="0.17000000000000001" stiffness="67000">
      <site site="tt_model_0/rod20_21/node_34" />
      <site site="tt_model_0/middle_platform/node_42" />
    </spatial>
  </tendon>
  <actuator>
    <motor name="tt_model_0/torque_x_ctrl" class="tt_model_0/" ctrllimited="true" ctrlrange="-100 100" gear="1" joint="tt_model_0/middle_platform/x_control" />
    <velocity name="tt_model_0/velocity_x_ctrl" class="tt_model_0/" ctrlrange="-100 100" joint="tt_model_0/middle_platform/x_control" kv="0" />
  </actuator>
  <sensor>
    <framepos name="tt_model_0/pos_outside_ball" objtype="body" objname="tt_model_0/middle_platform/" reftype="site" refname="world_site" />
    <framepos name="tt_model_0/pos_inside_ball" objtype="body" objname="tt_model_0/middle_platform/inside_ball" reftype="site" refname="world_site" />
    <frameangvel name="tt_model_0/angvel_outside_ball" objtype="body" objname="tt_model_0/middle_platform/" reftype="site" refname="world_site" />
    <frameangvel name="tt_model_0/angvel_inside_ball" objtype="body" objname="tt_model_0/middle_platform/inside_ball" reftype="site" refname="world_site" />
    <frameangacc name="tt_model_0/angacc_outside_ball" objtype="body" objname="tt_model_0/middle_platform/" />
    <frameangacc name="tt_model_0/angacc_inside_ball" objtype="body" objname="tt_model_0/middle_platform/inside_ball" />
    <accelerometer name="tt_model_0//unnamed_sensor_6" site="tt_model_0/middle_platform/inside_site" />
  </sensor>
  <keyframe>
    <key name="tt_model_0/initial_state" time="0" qpos="0.17941599999999999 0.17941799999999999 0.36503600000000003 0.97639500000000001 -5.8118600000000003e-05 9.1129099999999995e-05 0.21598999999999999 -0.17941599999999999 0.17941799999999999 0.36503600000000003 0.97639500000000001 -5.8118600000000003e-05 -9.1129099999999995e-05 -0.21598999999999999 -0.17941599999999999 -0.17941499999999999 0.36503600000000003 0.802284 1.5698399999999999e-05 -0.000106937 0.596943 0.17941599999999999 -0.17941499999999999 0.36503600000000003 0.802284 1.5698399999999999e-05 0.000106937 -0.596943 0.179421 1.72124e-06 0.54406100000000002 1 -2.9211200000000003e-11 2.6737199999999999e-07 -3.5272900000000002e-10 0.17941199999999999 1.7211999999999999e-06 0.185228 1 -4.24989e-11 2.4442400000000002e-07 4.32981e-10 -0.17941199999999999 1.7211999999999999e-06 0.185228 1 -4.2498700000000001e-11 -2.4442400000000002e-07 -4.3297100000000001e-10 -0.179421 1.72124e-06 0.54406100000000002 1 -2.9211300000000003e-11 -2.6737199999999999e-07 3.5273900000000001e-10 0 0.179422 0.54406100000000002 1 0 0 0 0 -0.179419 0.54406100000000002 1 0 0 0 0 -0.17940999999999999 0.185228 1 0 0 0 0 0.17941399999999999 0.185228 1 0 0 0 0 1.7212200000000001e-06 0.36463800000000002 1 -4.6430000000000001e-11 0 0 822.35299999999995 0 0" qvel="0 6.9729800000000005e-08 0 1.45504e-12 0 0.030700000000000002 0 6.9729800000000005e-08 0 1.42426e-12 0 -0.030700000000000002 0 6.9729800000000005e-08 0 -1.7240800000000001e-12 0 0.082785999999999998 0 6.9729800000000005e-08 0 -1.7047799999999999e-12 0 -0.082785999999999998 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 0 6.9729800000000005e-08 0 0 0 0 70 0 0" ctrl="-100 0" />
    <key name="tt_model_0/initial_state1" time="0" qpos="0.17932999999999999 0.22314100000000001 0.27458199999999999 0.92047199999999996 -0.38627699999999998 0.023045 0.054674199999999999 -0.17932999999999999 0.22314100000000001 0.27458199999999999 0.92047199999999996 -0.38627699999999998 -0.023045 -0.054674199999999999 -0.17941199999999999 -0.030902800000000001 0.52774100000000002 0.90754199999999996 -0.377382 -0.070750199999999999 -0.170131 0.17941199999999999 -0.030902800000000001 0.52774100000000002 0.90754199999999996 -0.377382 0.070750199999999999 0.170131 0.17941199999999999 0.22303200000000001 0.52774100000000002 0.92440599999999995 -0.381409 1.87586e-05 -3.10178e-06 0.17932999999999999 -0.031012100000000001 0.27458199999999999 0.92557 -0.37836599999999998 0.0117246 -0.00469259 -0.17932999999999999 -0.031012100000000001 0.27458199999999999 0.92557 -0.37836599999999998 -0.0117246 0.00469259 -0.17941199999999999 0.22303200000000001 0.52774100000000002 0.92440599999999995 -0.381409 -1.8764099999999999e-05 3.1040400000000001e-06 9.5790899999999993e-13 0.34996300000000002 0.40096199999999999 1 4.0099599999999997e-14 1.0991499999999999e-15 -7.3526199999999994e-12 -2.7757899999999998e-12 0.096064700000000003 0.65462500000000001 1 5.8886000000000003e-14 1.0504599999999999e-15 -7.3530599999999998e-12 -6.5096099999999998e-12 -0.157834 0.40096199999999999 1 -3.7627500000000002e-14 6.2732100000000002e-16 -7.3527099999999997e-12 -2.77591e-12 0.096064700000000003 0.147512 1 5.1108799999999998e-14 4.4750600000000003e-17 -7.35383e-12 -2.7758599999999998e-12 0.096064700000000003 0.40098400000000001 0.92388000000000003 -0.382683 2.81394e-12 -6.7931299999999998e-12 -349.07400000000001 2.3731399999999999e-11 -5.8805700000000003e-12" qvel="0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 70 0 0" ctrl="-100 0" />
  </keyframe>
</mujoco>
"""

In [18]:
m = mj.MjModel.from_xml_string(xml_string)
d = mj.MjData(m)